# Image Reduction Module

**Lecturer:** Dan Perley<br>
**Jupyter Notebook Authors:** Dan Perley, Kishalay De & Cameron Hummels

This is a Jupyter notebook lesson taken from the GROWTH Summer School 2019.  For other lessons and their accompanying lectures, please see: http://growth.caltech.edu/growth-school-2019.html

## Objective
Process raw images from a visible wavelength telescope and make them ready for photometric analysis

## Key steps
- Reduce raw images by applying biases & flats
- Align and combine consecutive images

## Required dependencies

See GROWTH school webpage for detailed instructions on how to install these modules and packages.  Nominally, you should be able to install the python modules with `pip install <module>` (or `pip3 install <module>`).  The external astromatic packages are easiest installed using package managers (e.g., `rpm`, `apt-get`).

### Python modules
* python 3
* astropy
* numpy
* matplotlib
* photutils
* pyregion

### External packages
* SExtractor https://www.astromatic.net/software
* SWarp https://www.astromatic.net/software (optional)


***

## Setup

### Import modules

In [ ]:
## Import necessary modules 

from astropy.io import fits
from astropy.wcs import WCS
from astropy.stats import sigma_clipped_stats
import glob
import os
import subprocess
import warnings
import numpy as np
import matplotlib.pyplot as plt
import photutils
import pyregion

# You can ignore any warnings that appear below, but if any modules can't be imported you need to install them

### Test dependencies

In order to complete the module some external software is required.  The following step checks that these are installed properly.

In [ ]:
def test_dependency(dep, alternate_name=None):
    """
    Test external dependency by trying to run it as a subprocess
    """
    try:
        subprocess.check_output(dep, stderr=subprocess.PIPE, shell=True)
        print("%s is installed properly as %s. OK" % (dep, dep))
        return 1
    except subprocess.CalledProcessError:
        try:
            subprocess.check_output(alternate_name, stderr=subprocess.PIPE, shell=True)
            print("%s is installed properly as %s. OK" % (dep, alternate_name))
            return 1
        except subprocess.CalledProcessError:
            print("===%s/%s IS NOT YET INSTALLED PROPERLY===" % (dep, alternate_name))
            return 0
    
dependencies = [('sextractor', 'sex'), ('SWarp', 'swarp')]
i = 0
for dep_name1, dep_name2 in dependencies:
    i += test_dependency(dep_name1, dep_name2)
print("%i out of %i external dependencies installed properly.\n" % (i, len(dependencies)))
if i != len(dependencies):
    print("Please correctly install these programs before continuing by following the instructions in README.md.")
else:
    print("You are ready to continue.")        

(SWarp is optional, but SExtractor is needed for astrometric alignment.  If you are unable to install it, it is possible to follow most of the module without it.)

### Define where data live

The various directories where data live are defined here for use later in this notebook.

In [ ]:
## Define data directories

curpath = os.path.abspath('.')                    # top level directory
dataFolder = os.path.join(curpath, 'data')        # data directory
biasFolder = os.path.join(dataFolder, 'bias')     # bias frames subdirectory
flatFolder = os.path.join(dataFolder, 'flat')     # flat fields subdirectory
sciFolder = os.path.join(dataFolder, 'science')   # science data subdirectory
procFolder = os.path.join(curpath, 'processing')  # processing directory
if not os.path.isdir(procFolder): 
    os.mkdir(procFolder)
else:
    for f in os.listdir(procFolder):
        try:
            os.remove(os.path.join(procFolder,f)) # clear the processing folder from previous iterations
        except:
            print('Could not remove',f)

Next we get lists of all the various file types, including lists and names of files we'll be writing out later.

In [ ]:
## Get file lists

os.chdir(sciFolder)
fileList = sorted(glob.glob('*.fits'))
os.chdir(curpath)
biasList = sorted(glob.glob(os.path.join(biasFolder,'*.fits')))
flatList = sorted(glob.glob(os.path.join(flatFolder,'*.fits')))
sciList = [os.path.join(sciFolder, file) for file in fileList]
procList = [os.path.join(procFolder, file).replace('.fits','.proc.fits') for file in fileList]
combinedFile = os.path.join(procFolder, 'AT2018cow_combined.fits')
resampledFile = os.path.join(procFolder, 'AT2018cow_resampled.fits')

print('Found',len(biasList),'bias files; ',len(flatList),'flat files; ',len(sciList),'science files')


***

## Preliminary Data Inspection and Exploration

Like nearly all astronomical images, our data are stored in the form of FITS files.   FITS files contain a plaintext header storing the observational metadata (e.g. the name of the telescope) plus an array of pixel data.  FITS files can may be 'simple' (single header, single pixel image) or may contain extensions (additional headers and/or pixel images).  

FITS files are loaded and examined using the astropy library in python.  Below we open up a FITS file and take a look at its structure.

In [ ]:
## Open an example FITS file

exampleFile = sciList[0]         # The first science image
HDUList = fits.open(exampleFile) # Open the file
HDUList.info()                   # Print some information about the FITS file structure.  

This reveals that the FITS header metadata is in extension 0 (the 'primary' header) and the pixel data is in extension 1.  The image dimensions are 2148x2501 pixels.


### Investigate the header

A good FITS header will contain almost everything we need to know about the observation.  Header data is stored as keyword + value pairs, optionally with a comment following the value.  Keywords are all-capitals and 8 characters or less; corresponding values can be much longer.  There is almost no standardization in the use of keywords at different telescopes, but OBJECT and EXPTIME are commonly used to store the name of the object being observe and the integration time in seconds, respectively.

Below we will load the header and print out a limited set of the header lines in "raw" format (showing how the header metadata is stored in the file), and also look up a few specific bits of information using their keywords.

In [ ]:
## Print some header properties

header = HDUList[0].header # Get the header out of the HDU list.
print(repr(header[0:13]))  # Print the first 14 lines of the header (in raw format).
print(repr(header[24:34])) # Print some additional lines
print(repr(header[48:52])) 
print(repr(header[67:71])) 
print(repr(header[129:135])) 
print()
print('The object is:', header['OBJECT'])   # Print the name of the object being observed
print('The filters are:', header['ACAMFILT']) # Print the names of the optical filters in the light path.
print('The exposure time is:', header['EXPTIME'])  # Print the exposure time

From this we can infer that this is a Sloan g-band image of AT2018cow and the exposure time was 90 seconds.  We can also see that this data was taken with the ACAM instrument on the WHT last July 14th (by your module leader).  This is only a select portion of the header; printing out the entire header would show other keywords relating to the detector settings and temperature and other properties of the instrument, telescope, and/or facililty.

### Examine the data

It's usually also a good idea to visually inspect the data to get a feel for its appearance and to make sure it doesn't have any obvious problems before we analyze it.  We will use the imshow command within pyplot, which requires us to provide some paramters defining the minimum and maximum range of the colorbar scale.  For this, we will use sigma-clipped image pixel statistics to come up with some reasonable values.

In [ ]:
## Get and display the image data

data = HDUList[1].data  # Get the data array (a simple numpy array) from the first extension.

mean, median, std = sigma_clipped_stats(data) # get some image statistics
plt.figure(figsize=(8,8))                     # set up the plot panel
plt.imshow(data, vmin = median - 2*std, vmax = median + 6*std, origin='lower')
plt.colorbar()
plt.show()

This shows us that the field of view of the instrument is circular and centered at approximately x=1000, y=1100.  A significant fraction of the CCD is not exposed.

While the image already looks pretty nice at a glance, raw data of this type is not suitable for anything other than crude preliminary analysis: the sensitivity may vary across the field, it may contain instrumental signals from the detector electronics, it may be contaminated by cosmic rays, etc.  Removing these effects to produce a "pure" image of the night sky is the role of data reduction.  There are two generic steps (bias correction and flat-fielding), which can be followed by a variety of possible high-level calibration and artifact-correction steps.  We will address these in the next section.  For now, we are done inspecting the science image so we will close it.

In [ ]:
HDUList.close()                  # Close the file (good practice, but not strictly necessary)

***

## Creating a bias frame

The detector image comes with an electronic offset introduced by the voltages applied to the the semiconductor-based detector.   This can be clearly seen in the science image above: even the unexposed parts of the detector have an apparent signal level of about 1900 counts.  We have to remove this offset to get real counts received by the detector. 

Different bias-removal methods are employed by different detectors.  In most cases, we record a set of images ("bias frames") with zero exposure time, so that the recorded image reflects *only* the intrinsic offsets in the detector (the detector does not receive any photons if the exposure time is zero).  To reduce detector noise and to remove cosmic ray contamination, we usually take many bias frames and average them together using median-combination.  (Even though the exposure time is zero, cosmic rays can strike the detector during readout.)

### Investigate the data

As always, it's a good idea to inspect your images before you process them.  As an exercise below, open the first bias image and verify that it doesn't contain any obvious astronomical signals.

#### Exercise 1: Open and display the first bias image, and determine its clipped standard deviation.

In [ ]:
## EXERCISE 1:  Display the first bias image and determine its clipped standard deviation.

biasFile = biasList[0]         # The first science image
HDUList = fits.open(biasFile) # Open the file

data = HDUList[1].data  # Get the data array (a simple numpy array) from the first extension.

mean, median, std = sigma_clipped_stats(data) # get some image statistics
plt.figure(figsize=(8,8))                     # set up the plot panel
plt.imshow(data, vmin = median - 2*std, vmax = median + 6*std, origin='lower')
plt.colorbar()
plt.show()

print('Standard deviation:', std)

### Median-combine the bias images

This process could be repeated with the other four images (in Python or using external software like DS9), but we'll trust the data for now and proceed to the next step, which is to combine all five bias frames together.  To do this, we set up a 3D numpy array and put a bias frame in each layer, and then median-combine along the 3rd dimension to produce a single, median-combined image (our "master bias").

In [ ]:
## Median-combine the bias frames to make a master bias

# Create a 3D array to store all the bias files together.
ny = 2501
nx = 2148
numBiasFiles = len(biasList)
biasImages = np.zeros((ny, nx, numBiasFiles))

# Add the files to the array
for i in range(numBiasFiles):
        HDUList = fits.open(biasList[i])     # Open the file
        biasImages[:,:,i] = HDUList[1].data  # Load the data into the appropriate layer
        HDUList.close()                      # Close the file

# Create the master bias frame by doing a median combination for each pixel over all layers
masterBias = np.median(biasImages, axis=2)


#### Exercise 2: Display the master bias image and check that the noise is reduced compared to a single bias frame by calculating and printing out its (sigma- clipped) standard deviation.

In [ ]:
## EXERCISE 2:  Display the master bias image and calculate its clipped standard deviation.

data = masterBias

mean, median, std = sigma_clipped_stats(data) # get some image statistics
plt.figure(figsize=(8,8))                     # set up the plot panel
plt.imshow(data, vmin = median - 2*std, vmax = median + 6*std, origin='lower')
plt.colorbar()
plt.show()

print('Standard deviation:', std)


It should look pretty similar to the individual bias frame, but with lower noise.  Read noise is generally negligible compared to the sky noise for broadband imaging observations such as this one, so this isn't a major consideration here.  However, for narrowband observations or for spectroscopy this may not be the case, so it's always a good idea to take many bias frames if possible.

***

## Creating a flat field

The image recorded on the detector $I(x,y)$ is a product of the true brightness distribution on the sky $S(x,y)$ and the response of the telescope optics and detector in different parts of the image $F(x,y)$. with any bias signal added: i.e.: 

$I(x,y) = F(x,y) S(x,y) + B(x,y)$.  

We already have $B(x,y)$ thanks to our bias frames, but we also need to determine $F(x,y)$ to reconstruct $S(x,y)$.

To do this, we expose the detector to a uniform light source -- this can either be a uniformly illuminated part of the telescope dome or the twilight sky (the assumption is that the sky is uniformly bright over a small field of view).

### Investigate the data

As usual it's good to start by looking at the data.

#### Exercise 3: Open and display the first flat-field image.  Set vmin and vmax appropriately to be sure you can see the structure in the flat-field.

In [ ]:
## EXERCISE 3:  Display the first flat-field image.  

flatFile = flatList[0]         # The first flat image
HDUList = fits.open(flatFile) # Open the file

data = HDUList[1].data  # Get the data array (a simple numpy array) from the first extension.

mean, median, std = sigma_clipped_stats(data) # get some image statistics
plt.figure(figsize=(8,8))                     # set up the plot panel
plt.imshow(data, vmin = 28000, vmax =35000, origin='lower')
plt.colorbar()
plt.show()


We won't look at the other images, but it's a good idea to scan through the files to make sure that the configuration is what we expect and that the frames aren't saturated.  In the next exercise you will write a loop to do this check.

#### Exercise 4: Verify that all five flat-field images are acceptable by looping over each file and printing out the filter (ACAMFILT keyword), the exposure time (EXPTIME keyword), and the median pixel value (calculated from the data).

In [ ]:
## EXERCISE 4: Verify that all five flat-field images are acceptable by printing out some basic properties.

for fileName in flatList:
    HDUList = fits.open(fileName)
    header = HDUList[0].header # Get the header out of the HDU list.
    print('The filters are:', header['ACAMFILT']) # Print the names of the optical filters in the light path.
    print('The exposure time is:', header['EXPTIME'])  # Print the exposure time
    data = HDUList[1].data
    print(np.median(data))
    HDUList.close()

The saturation level of a typical CCD is around 60000, so these values all look good.  They also all employed the same filter as for our science observations.

### Normalize and combine flat-fields

The master flat field is formed by median combination in a similar way to the master bias frame, above.  There are two big differences.  First, we have to remove the bias from each image.  Second, we have to adjust for the fact that the light level may be changing from image to image by measuring the average pixel level in each image and dividing the image by it (normalizing).

It is important that this normalization level be calculated only with pixels for which there is actually some signal.  For this camera much of the CCD is unused and has no signal, so we will use a square subset of the image and calculate its median.  Then we divide by this factor and stick it into the appropriate layer of a 3D array.  The median combination at the end is left an an exercise for you.

In [ ]:
## Load and normalize the flat fields in preparation for making a master flat field.

# Set up the 3D array
numFlatFiles = len(flatList)
flatImages = np.zeros((ny, nx, numFlatFiles))

# Load the files into the array, with bias subtraction and normalization
for i in range(numFlatFiles):
        # Load the data from the fits file
        HDUList = fits.open(flatList[i])
        data = HDUList[1].data * 1. # Convert to floating point
        HDUList.close()

        # Bias-subtract, normalize, and add to the array layer
        data -= masterBias
        normfactor = np.median(data[500:1600,500:1600])
        print(normfactor)
        flatImages[:,:,i] = data / normfactor


#### Exercise 5: Perform the median combination to produce a 2D array called masterFlat, and display it. (Be smart with the scaling: use only the exposed area to calculate the stats for vmin/vmax.)

In [ ]:
## EXERCISE 5: Median-combine the flat field layer array into a master flat and display it.

# Create the master bias frame by doing a median combination for each pixel over all layers
masterFlat = np.median(flatImages, axis=2)

data = masterFlat
mean, median, std = sigma_clipped_stats(data[500:1600,500:1600]) # get some image statistics
plt.figure(figsize=(8,8))                     # set up the plot panel
plt.imshow(data, vmin =median-5*std, vmax =median+5*std, origin='lower')
plt.colorbar()
plt.show()



### Flag unexposed pixels

Note that large fraction of the above image has no signal at all, as you can easily check by changing the scaling above to e.g. vmin=0, vmax=1.1.  Dividing no-signal values by very low flat-field values will fill the unexposed part of the detector with garbage values, which isn't desirable.   Instead, we will set all the low pixel values to NaN ("not a number"), which signifies that these pixels have no scientific value.  These NaN values will propogate to the science images after flat-field division and flag the unwanted pixels in those as well.

In [ ]:
## Set the unexposed pixels to NaN, and display again

masterFlatFixed = np.copy(masterFlat) # Use 'copy' to preserve the original masterFlat
if np.any(masterFlat < 0.2):
        # Set all flat-field values lower than 0.2 to NaN
        masterFlatFixed[masterFlat < 0.2] = float('NaN') 

plt.figure(figsize=(8,8))
#scale the image according to its statistics
plt.imshow(masterFlatFixed, vmin = 0, vmax = median + 5*std, origin='lower')
plt.colorbar()
plt.show()

At this point, we have the necessary calibration images to reduce science images and visualize the true brightness distribution on the sky.  Remember that the observed intensity on the detector is:

$I(x,y) = F(x,y) \,  S(x,y) + B(x,y)$

We now have to retrieve $S(x,y)$ from $I(x,y)$, by inverting this equation, i.e.:

$S(x,y) = \frac{I(x,y)-B(x,y)}{F(x,y)}$

## Pre-processing science frames

Now we will apply the bias flat calibrations to the science frames and write processed science frames to disk for further processing.  To simplify matters, we will write these processed files in 'simple' FITS format, combining the primary header data with the (bias- and flat-corrected) image data in the FITS extension for each file.

In [ ]:
## Bias subtract and flat-field all science frames and write out pre-processed files.

numSciFiles = len(sciList)
print('Found %d science files'%numSciFiles)

for i in range(numSciFiles):
        # Read in the FITS data.
        HDUList = fits.open(sciList[i])
        primaryHeader = HDUList[0].header
        imageData = HDUList[1].data    
        HDUList.close()
        
        # Correct for the bias and flats here
        procData = (imageData - masterBias) / masterFlatFixed
        
        # Prepare the output FITS structure in simple format
        procHDU = fits.PrimaryHDU(procData)  # Create a new HDU with the processed image data
        procHDU.header = primaryHeader       # Copy over the header from the raw file
        procHDU.header.add_history('Bias corrected and flat-fielded') # Add a note to the header

        # Write the reduced frame to disk
        print(sciList[i],'->',procList[i])
        procHDU.writeto(procList[i], overwrite=True)
        

Now loop through the processed science images we just wrote to the disk and plot them to see what they look like.

In [ ]:
# Inspect the processed science files

plt.figure(figsize=(14,14))
for i in range(numSciFiles):
        procHDU = fits.open(procList[i])
        procData = procHDU[0].data
        procHDU.close()
        
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            mean, median, std = sigma_clipped_stats(procData)
        plt.subplot(2,2,1+i)
        plt.imshow(procData, vmin = median - 2*std, vmax = median + 10*std, origin='lower')
        plt.colorbar()


These images are suitable for some types of scientific analysis.  However, we would also like to combine all three together to remove cosmic rays and reduce the noise (go deeper).  This is not as simple as combining the biases or the flats: the images were dithered (the telescope was moved between each exposure) so the astronomical sources lie at different pixel positions in different images.  We will have to align the images relative to each other before stacking.

## Relative astrometric aligment

Aligning images is a form of astrometry (the measurement of precise positions of objects on the sky).  We will worry about absolute astrometry versus the all-sky RA, DEC reference system later, but we do need a way to measure and remove the relative offsets between images now.  If our image-to-image dithering motions are small, the telescope field of view does not contain significant distortions, and the instrument was not rotated, we can do this ourselves in Python by measuring the relative positions of a reference star from image to image and offsetting.

### Measure reference star locations

The first step is to find a nice reference star near the center of the image.  By measuring its centroid as it shifts between all three images, we can calculate how much to offset each image to line them up with each other.  Fortunately there is an excellent, nonsaturated star just to the left of the bright galaxy that's well-suited for this purpose.  Its approximate positions (x,y) in the three science images, estimated by eye, are (998,1161), (998,1121), and (958,1121) respectively.  But it would be better to measure the position more precisely, so the below loop takes a little cutout around our rough position in each image, calculates the centroid, and stores that in a separate list.

In [ ]:
## Measure the precise position of a star in each image

# Set the approximate star coordinates
estimated_star_x = [998,998,958]
estimated_star_y = [1161,1121,1121]
actual_star_x = [0,0,0] # dummy values
actual_star_y = [0,0,0]

for i in range(numSciFiles):
        HDUList = fits.open(procList[i])
        procData = HDUList[0].data
        HDUList.close()
        (x0, x1) = (estimated_star_x[i]-20, estimated_star_x[i]+20)
        (y0, y1) = (estimated_star_y[i]-20, estimated_star_y[i]+20)
        cutout = procData[y0:y1,x0:x1]
        mean, median, std = sigma_clipped_stats(cutout)
        cx, cy = photutils.centroid_com(cutout-median)
        actual_star_x[i] = cx+x0
        actual_star_y[i] = cy+y0
        print('Image', i, ':', actual_star_x[i], actual_star_y[i])
        

### Apply offsets and crop/shift images

Next we apply these offsets to the image.  In the general case (e.g. if rotations are involved or if the pixel offsets are not pure integers), this involves resampling the image along a new coordinate grid.  To keep things simple, we are just going to apply a simple integer offset by taking appropriately matching slices of each array.  The first image will be cropped from y=150:2150 and x=150:2015 (2000 pixels on each axis).  The other two images will be cropped similarly but with the x and y starting and ending values changed as needed based on their offset.

While we're at it, we also subtract the sky background: because the sky level can vary, it's important to remove it prior to stacking the images.

In [ ]:
## Offset each image by subsetting to line them up with each other, and subtract the sky

# Define the default crop region and set up the 3D array
xmin = 50
xmax = 2050
ymin = 150
ymax = 2150       
alignedImages = np.zeros((ymax-ymin,xmax-xmin, numSciFiles))

# Calculate and apply offsets, and load the 3D array with appropriately shifted data
for i in range(numSciFiles):
        # Calculate the offset relative to the reference image
        xoffset = int(round(actual_star_x[i]-actual_star_x[0]))
        yoffset = int(round(actual_star_y[i]-actual_star_y[0]))
        print('Image', i, 'offset:', xoffset, yoffset)

        # Open the FITS file and get the data
        HDUList = fits.open(procList[i])        
        procData = HDUList[0].data
        HDUList.close()
        
        # Crop the data appropriately to match up with the reference image
        shiftData = procData[ymin+yoffset:ymax+yoffset, (xmin+xoffset):(xmax+xoffset)]
        shiftData -= np.nanmedian(shiftData) # Subtract sky
        alignedImages[:,:,i] = shiftData     # Add this layer to the 3D array
        
# Verify the images are aligned by plotting a little cutout around where the star should be
for i in range(numSciFiles):
        plt.subplot(1,3,1+i)
        (starX, starY) = (int(round(actual_star_x[0]-xmin)), int(round(actual_star_y[0]-ymin)))
        starCutout = alignedImages[starY-9:starY+9,starX-9:starX+9,i]
        plt.imshow(starCutout, vmin = 0, vmax = 10000, origin='lower')        
        


The star is now in the same place (same pixels) on all three images, so that's good.

### Median-combine science images

Finally we median-combine the aligned images together (in case you haven't noticed, optical astronomers are really into medians).  Note that while we have subtracted the sky, we have not scaled the data, so we are assuming all three images have the same exposure time and very similar sky conditions (no clouds).  In variable seeing or transparency a median combination will produce strange results!

#### EXERCISE 6: Median-combine the shifted/cropped science images and save the result to disk.   For the output filename, use the string stored in combinedFile.  (Hint: use nanmedian for the stack, and re-use code from when the processed files were written to disk)

In [ ]:
## EXERCISE 6: Median-combine the shifted/cropped science images and save the result to disk.

combinedImage = np.nanmedian(alignedImages, axis=2)

HDUList = fits.open(procList[0])
primaryHeader = HDUList[0].header
HDUList.close()

# Prepare the output FITS structure in simple format
procHDU = fits.PrimaryHDU(combinedImage)  # Create a new HDU with the processed image data
procHDU.header = primaryHeader       # Copy over the header from the raw file
procHDU.header.add_history('Stacked') # Add a note to the header
procHDU.writeto(combinedFile, overwrite=True)
        

If this worked correctly and you wrote the file out to the path given in the combinedFile variable, the code below should display it for you:

In [ ]:
## Confirm the stack worked correctly by reloading the combined image from disk and displaying it.

HDUList = fits.open(combinedFile) # Open the first science file in order to retrieve its header
combinedImage = HDUList[0].data
HDUList.close()

plt.figure(figsize = (12,10))
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    mean, median, std = sigma_clipped_stats(combinedImage)
plt.imshow(combinedImage, vmin = median - 1*std, vmax = median + 100*std, origin='lower')
plt.colorbar()
plt.show()


## Absolute astrometric calibration

We now have a combined, science-ready image... sort of.  In reality, doing any sort of scientific analysis on this image as it stands will be a big pain, because it doesn't contain any positional information.  If we wanted to extract information about a particular star or galaxy, or (worse) a whole list of objects, this would be extremely frustrating: we would have to scan the image by eye to find each object, write down its pixel coordinates, and send them one-by-one to our photometry or other analysis code.

Fortunately, the FITS format can encode astrometric (positional) information via the header, enabling us to transform (x,y) to (RA,Dec) and back again for any location in the image.  Basic header keywords encode the position, pixel scale, and rotation of the image (plus the spherical projection to be used); these are fully standardized and supported by nearly all astronomical software.  Additional keywords provide distortion terms to allow for deviations from perfectly rectilinear CCD/optics, although unfortunately there are several conflicting conventions for treating distortions and there is no guarantee that your software will support them correctly.

Solving astrometry requires matching a group of stars detected in your image to the stars in the catalog, and solving (fitting) for the parameters in the system of astrometric equations that relate pixel coordinates (x,y) to world coordinates (RA,Dec).  Doing this by hand is painstaking so we will rely on a general-purpose python script that pattern-matches groups of stars in the image to stars in sky catalogs, solves for the astrometric parameters, and writes them to the header.  This process is entirely automated: all we have to give the code is the pixel scale (the projected "size" of each pixel in arcseconds), which for WHT/ACAM is 0.253.

The code requires SourceExtractor (aka SExtractor), an external piece of software that automatically detects and measures the properties of all star-like objects in the image.  Note that you may have to modify the "autoastrometry3.py" file if this is called in an unusual way from your operating system.

In [ ]:
## Astrometrically calibrate the reduced image using an external python script

autoastrometry_script = os.path.join(dataFolder, 'autoastrometry3.py')
os.chdir(procFolder)
sexcommand = 'sextractor' # Change this if you get an error below

try:
    #Run the autoastrometry script using 2MASS as the reference catalog by specifying 'tmc' with the '-c' option
    astromFile = combinedFile.replace('.fits','.astrom.fits')
    command = 'python3 %s %s -c sdss -px 0.253 -inv -o %s -sex %s -1' % (autoastrometry_script, combinedFile, astromFile, sexcommand)
    print('Executing command: %s' % command)
    print('Processing...')
    rval = subprocess.run(command.split(), check=True)
    print('Process completed.  See your terminal window to check output.')
except subprocess.CalledProcessError as err:
    print('Could not run autoastrometry with error %s. Check if file exists.'%err)
    
os.chdir(curpath) 

If this worked you should see a list of successful matches in your terminal followed by an information about the applied rotation and spatial offset.  To make sure it works, we can load the star positions (in world coordinates, i.e. RA/Dec) on top of our image:

In [ ]:
## Display astrometrically calibrated image with star catalog overlaid

# Open the file and get the info
HDUList = fits.open(astromFile)
header = HDUList[0].header
combinedImage = HDUList[0].data
HDUList.close()

# Display the image, associating the WCS information with the axes
wcs = WCS(header)
fig = plt.figure(figsize=(14,14))
ax = plt.subplot(projection=wcs)
plt.imshow(combinedImage, vmin=0, vmax=1000, origin="lower")

# Load star catalog region file
regionFile = os.path.join(procFolder,'cat.wcs.reg')
r = pyregion.open(regionFile).as_imagecoord(header=HDUList[0].header)
patch_list, artist_list = r.get_mpl_patches_texts()

# Overlay the star catalog on the plot
for p in patch_list:
    ax.add_patch(p)
for t in artist_list:
    ax.add_artist(t)
plt.show()

Most of the catalog star positions should land on top of bright stars in the image, indicating that our image is correctly aligned to at least a reasonable approximation.  Small deviations may be due to proper motions (stars move!) and catalog errors, or due to distortions in our image field that we have not tried to solve for.

Despite the lack of distortion solution, this should be good enough to find any star or object in the field automatically given its coordinates.  Also, once an image is approximately aligned applying further corrections (using more sophisticated software like scamp) is much easier.

You can load this image in DS9 or other viewing software to interactively explore it in more detail.

## Resampling

The above image does not have North up and East to the left, the normal convention for displaying astronomical images.  The easiest way to project an image to the standard reference system is to resample it using SWarp, another piece of external code.  This can be done as follows:

In [ ]:
## Resample the image to standard orientation using SWarp

swarpConfigFile = os.path.join(dataFolder, 'stack.swarp')

try:
    #IMAGEOUT_NAME is the output stacked file, RESAMPLE_DIR is the directory where the resampled images are to be stored

    command = 'swarp %s -c %s -IMAGEOUT_NAME %s -RESAMPLE_DIR %s' % (astromFile, swarpConfigFile, resampledFile, procFolder)
    print("Executing command: %s" % command)
    print("Processing...")
    rval = subprocess.run(command.split(), check=True)
    print("Process completed.")
except subprocess.CalledProcessError as err:
    print('Could not run swarp. Can you run it from the terminal?')



This can then be displayed as normal:

In [ ]:
## Display the resampled image

HDUList = fits.open(resampledFile)
wcs = WCS(HDUList[0].header)
resampledData = HDUList[0].data
HDUList.close()

fig = plt.figure(figsize=(14,14))
ax = plt.subplot(projection=wcs)
plt.imshow(resampledData, vmin=0., vmax=1000, origin="lower")


Note that SWarp has set the NaN values to zero, which is somewhat problematic: we may want to change these in software in the future before doing any analysis.  The noise properties have also been changed by the resampling/interpolation.

SWarp also supports resampling a series of (astrometrically-corrected) images onto a common grid and median-stacking them to produce a combined file: simply give it a list of files instead of a single file.  If you have extra time, you can try to produce a stacked image using an this method instead of our Python-based method: astrometrically calibrate the three individual images (with autoastrometry.py) and stack them (with SWarp).  This can be done entirely from the UNIX command line outside of the notebook if you prefer.

## Concluding remarks

At this point, we're done!  Either of the above images could be used for photometry or for other forms of image analysis that will you learn about in the next session.

It is important to note that procedures may differ from telescope to telescope and depending on the type of analysis you're interested in.  Depending on your detector, filter passband, the number of exposures, the time-variability of your source, whether the moon is up or down, whether clouds were present, and the specifications of the type of science you need to do, you may need to carry out additional steps or different steps, such as:

* Overscan subtraction
* Dark current subtraction
* Bad pixel masking
* Source masking in sky flat construction
* Super-sky flat construction and subtraction
* Linearity correction
* Fringe correction or subtraction
* Median-filtered or model-based sky subtraction
* Dithered-group sky subtraction
* Cosmic ray filtering/masking
* Photometric scaling of individual exposures
* Astrometric distortion corrections

### Cleanup

To save disk space, you can execute the cell below to remove your processed data.

In [ ]:
# WARNING: WILL REMOVE YOUR PROCESSED DATA FROM THIS NOTEBOOK
os.chdir(curpath)
for f in os.listdir(procFolder):
    try:
        os.remove(os.path.join(procFolder,f)) # clear the processing folder from previous iterations
    except:
        print('Could not remove',f)